### 年収を平均化

In [7]:
import pandas as pd
import numpy as np

In [4]:
file_path = "/Users/lex/CodeProjects/MyProject/Mitaron/Average Annual Income Health Insurance List.csv"
df = pd.read_csv(file_path)

In [10]:
df.head()

,保険者番号,保険者名,平均年収 (万円)
0,6010334,北海道新聞社健康保険組合,730
1,6010391,北海道農業団体健康保険組合,600
2,6010516,北海道電力健康保険組合,740
3,6010532,北洋銀行健康保険組合,700
4,6010615,北海道銀行健康保険組合,690


In [15]:
conditions = [
    df["平均年収 (万円)"].between(500, 600, inclusive="left"),
    df["平均年収 (万円)"].between(600, 650, inclusive="left"),
    df["平均年収 (万円)"].between(650, 700, inclusive="left"),
    df["平均年収 (万円)"].between(700, 750, inclusive="left"),
    df["平均年収 (万円)"].between(750, 800, inclusive="left"),
    df["平均年収 (万円)"].between(800, 850, inclusive="left"),
    df["平均年収 (万円)"].between(850, 900, inclusive="left"),
    df["平均年収 (万円)"].between(900, 1000, inclusive="left"),
    df["平均年収 (万円)"].between(1000, 1250, inclusive="left"),
    df["平均年収 (万円)"] >= 1250
]
choices = ["500 ~ 600", "600 ~ 650","650 ~ 700", "700 ~ 750","750 ~ 800", "800 ~ 850","850 ~ 900", "900 ~ 1000","1000 ~ 1250", "1250 `"]

df["平均(荒)"] = np.select(conditions, choices, default = "D")
display(df.head(50))

,保険者番号,保険者名,平均年収 (万円),平均(荒)
0,6010334,北海道新聞社健康保険組合,730,700 ~ 750
1,6010391,北海道農業団体健康保険組合,600,600 ~ 650
2,6010516,北海道電力健康保険組合,740,700 ~ 750
3,6010532,北洋銀行健康保険組合,700,700 ~ 750
4,6010615,北海道銀行健康保険組合,690,650 ~ 700
5,6010649,北海道信用金庫健康保険組合,630,600 ~ 650
6,6010656,栗林商会健康保険組合,680,650 ~ 700
7,6010698,北海道通運業健康保険組合,650,650 ~ 700
8,6010722,楢崎健康保険組合,670,650 ~ 700
9,6010813,エア・ウォーター健康保険組合,900,900 ~ 1000


In [16]:
df.to_csv("平均年収最終.csv", index = False, encoding = "utf-8-sig")